In [22]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.restaurants

In [64]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

def convertRestaurant(restaurant):
    return {
        "name": restaurant["name"],
        "cuisine":restaurant["cuisine"],
        "position": toGeoJSON(restaurant["address"]["coord"])
    }

cursor = db.restaurants.find({"address.coord":{"$size":2}},{"_id":0})
geoRestaurants = list(map(lambda r: convertRestaurant(r),cursor))


In [65]:
import pandas as pd

df = pd.DataFrame(geoRestaurants)
df.head()

,cuisine,name,position
0,Hamburgers,Wendy'S,"{'type': 'Point', 'coordinates': [-73.961704, ..."
1,Irish,Dj Reynolds Pub And Restaurant,"{'type': 'Point', 'coordinates': [-73.98513559..."
2,American,Riviera Caterer,"{'type': 'Point', 'coordinates': [-73.98241999..."
3,Jewish/Kosher,Tov Kosher Kitchen,"{'type': 'Point', 'coordinates': [-73.8601152,..."
4,American,Brunos On The Boulevard,"{'type': 'Point', 'coordinates': [-73.8803827,..."


In [68]:
df.to_json("rest_with_coords.json", orient="records", lines=True)

In [130]:
cursor = db.clean.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [-74.1080159, 40.7046139]
      },
      "$minDistance": 0,
      "$maxDistance":10000
    }
  }
})

import pandas as pd
df = pd.DataFrame(list(cursor))
df.head()

,_id,cuisine,name,position
0,5c6692ad7cb9799ee5610be8,Italian,Blue,"{'type': 'Point', 'coordinates': [-74.1077385,..."
1,5c6692ad7cb9799ee5610882,Italian,Crispy Pizza,"{'type': 'Point', 'coordinates': [-74.0982604,..."
2,5c6692ad7cb9799ee560e28d,American,Ruth M Liedys Shore Inn,"{'type': 'Point', 'coordinates': [-74.097056, ..."
3,5c6692ad7cb9799ee56117d1,American,The Cuckoo'S Nest,"{'type': 'Point', 'coordinates': [-74.097927, ..."
4,5c6692ad7cb9799ee560e613,Chinese,D & B Double Chinese Restaurant,"{'type': 'Point', 'coordinates': [-74.0861942,..."


In [131]:
coords_df = pd.DataFrame(pd.DataFrame(df["position"].values.flatten().tolist())["coordinates"].values.tolist())
clean_df = pd.concat([df,coords_df],axis=1)
clean_df.drop(['_id',"position"], inplace=True, axis=1)
clean_df = clean_df.rename({0:"long",1:"lat"}, axis=1)


In [132]:
clean_df.to_json('visualize_restaurants.json', orient="records")